In the following, the code cells are the prototypes of the code we need. What the user would actually have to type is shown in the text cells. 

Ideally, go so far as to prototype it, making functions that hardwire the notebook examples so you can see how it would work. Think about how it could work not as below with astroquery.heasarc.list_sia_services(), for example, but just as astroquery.list_sia_services(). The whole point is the user should care which site it comes from.

If HEASARC takes over astroquery.heasarc, we could use things like:

TAP:  

    astroquery.list_image_services(source='heasarc') 

TAP:  

    astroquery.surveys_like("Redshift”,source='heasarc')

SIA:  
    
    astroquery.get_image() # skyview vs xamin? 


 notes on Vandana’s collected workshop notebook:


In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display

## For handling ordinary astropy Tables
from astropy.table import Table

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

General use regtap function for use internally only, i.e. users shouldn't need to see this:

In [18]:
def regtap_query_string(service_type, source='',name_like=''):
    ''' Wrapper for RegTAP queries, returns short_name, res_description, and access_url
   
    This should have a lot of options. Search for wavebands etc.?   
    '''
    criteria="where a.cap_type='{}'".format(service_type)    
    if source is not '':
        criteria=criteria + " and a.ivoid like 'ivo://%{}%'".format(source)
    if name_like is not '':
        criteria=criteria + " and b.short_name like %{}%".format(name_like)
    return {
        "request":"doQuery",
        "lang":"ADQL",
        "query":"""
            select b.short_name,b.res_description, c.access_url 
            from rr.capability a natural join rr.resource b natural join rr.interface c
            {} 
            order by short_name;
        """.format(criteria)
        }

def request2table(url,data,params):
    """Just wraps the requests.get() and converts to astropy table"""
    r=requests.get(url,data=data,params=params)
    return Table.read(io.BytesIO(r.content))

def regtap_get_service( source='' , service_type='' ):
    """Query regtap for a service of the given type and optional source."""
    regtap_url='http://vao.stsci.edu/RegTAP/TapService.aspx/sync'
    if 'image' in service_type:
        service_type='simpleimageaccess'
    elif 'cone' in service_type: 
        service_type="conesearch"
    elif 'spectr' in service_type:
        service_type='simplespectralaccess'
    else:
        print("ERROR: don't recognize the requested service type")
        raise
        
    # Warning:  'query' is bizarrely case-sensitive:
    tap_params = astroquery.regtap_query_string(service_type, source )
    return astroquery.request2table( regtap_url, data=tap_params )



Combining the columns from the different services is again annoying. Need to use a function that combines them using the UCD? Eg., in some places HEASARC uses the column name “sia_url”, and this is the UCD “VOX:Image_AccessReference”, so something inside would have transparently translate from each different archive’s column names to something standardized. (The UCD itself? Or something we define, e.g., “image_url”?)

We should just use UCDs or UTYPEs and even just **ignore** all column names that we get back and immediately rename everything with the UCD and use only that internally. Then to hand it back to the user, we define our own astroquery column names that are obvious, like “URL”, etc., though storing the history (the UCD and the original service-specific column name) in the column’s meta data.

We need to keep the ucd information around, however. It's in the VOTable XML that we should get back from any service. And it IS kept in the astropy table as, e.g., uvot_table['Ra'].meta['ucd'] or uvot_table['URL'].meta['ucd']. So we can use these.

So under the hood, we would take any table we get back and rename each column with its UCD. Then to give things to the user, define simple dictionaries of the common columns such as:

In [23]:
# Somehow have this overloaded for objects of type SIA, SSA, etc.? Or just one list if there are no ambiguities? 
def ucd2col(ucd):
    u2c={
        "VOX:Image_AccessReference":"URL",
        "meta.ref.url":"URL"
    }
    return u2c[ucd]
    
def col2ucd(col):
    # But these aren't unique. 
    c2u={
        "URL":["VOX:Image_AccessReference","meta.ref.url"]        
    }
    return c2f[col]
    
    
def standardize( list_of_tables):
    """Take a list of astropy tables that have all different columns and convert to one standard astropy table
    
    Use the list_of_tables[table_number][column].meta['ucd'] to identify the standard columns we want.
    
    """
    return astropy_table, metadata

3 NED example cone search:  

Instead of searching NED ‘manually’, a generic cone search that you can give a list of ras, decs, and radii (or just one obviously) and optionally specify that you want ‘ned’ results or some other IVOID substring. If you ask for known things like ‘ned’, it has hard-wired (?) base URL and query path. If not, it queries the RegTAP to find out what cone searches are available that match a given ivoid_string.  So the user would call:

    cone_results, meta = astroquery.cone_search(ras, decs, radii, [source=’some_ivoid_string_eg_ned']) 

where ras, decs, and radii can be floats or arrays of floats, get back table of objects for a specified source or list of tables, one for each matching source? Since every table will return different columns, need to return some kind of meta data result as well. Separate object or attached to each result column's meta data? 

Started to define a function for the second part of this cell in the workshop notebook that got the pass bands from NED. This is very NED-specific. Any way to generalize?

    ned_info = astroquery.get_ned_info( ra, dec, radii )

calls the cone search and passes the ACREF for each match to NED again to get the info. But ACREF isn't a required value returned by a cone search. All that's required is the ID, RA, and DEC. So I don't think this can be generalized.


In [20]:

def one_cone_search( ra, dec, radius, source):
    params = {'RA': RA, 'DEC': DEC, 'SR':radius}
    r=requests.get(service,params=params)
    return Table.read(io.BytesIO(r.content))

def cone_search( ras, decs, radius, source='' ):
    """ This is based on the AAS workshop example. Haven't yet looked at the vo_conesearch() that exists, etc."""
    # Define a list of URLs to query. 
    services=astroquery.regtap_get_service(source=source,type="cone")
    # For each service, the returned columns are different. 
    # So cannot make one big astropy table, instead start with a list 
    # of tables, one per service.
    all_results=[]
    for service in services:
        # Start an astropy table for this service? Not sure of the initialization
        service_results=astropy.Table()
        for i in range(0,len(ras)):
            # Not sure of the extension. 
            service_results.extend( astroquery.one_cone_search(ras[i],decs[i],radius,service) )
            # Get some meta data.
        # Now that we've done all the searches for this service, append this one astropy table to the *list*
        #  of astropy tables in the results.
        all_results.append(service_results)
        
    return astroquery.standardize(all_results) 



4 TAP:  

    sia_services = astroquery.list_sia_services( [source=’ivoid_string_eg_heasarc'] , [name_like=‘allwise’], [description_like=‘whatever’] ) 

This one can easily be generalized so you can get images from any service (or a chosen one) that have a short name substring, or description substring. (Or perhaps just string_like=‘whatever’ and it searches both short name as well as description?) It returns a table of information, including the ‘access_url’ that you can then plug into another generic function




In [22]:
def list_image_services( source='', name_like=''):
    service=astroquery.regtap_get_service(service_type="image",source=source,name_like=name_like)
    return request2table(url=service,)
def list_spectra_serices():
    service=astroquery.regtap_get_service(service_type="spectra",source=source,name_like=name_like)
    return request2table(url=service,)
def list_cone_serices():
    service=astroquery.regtap_get_service(service_type="cone",source=source,name_like=name_like)
    return request2table(url=service,)


5 SIA

Then pick one of the listed services (say number 20, after you looked at the descriptions) and query it to get an image.

    image_url = astroquery.get_image_url( access_url=sia_services[20][‘access_url’], pos=‘185.47873,4.47365', size=‘0’]

or perhaps you don't know which service is quite what you want, so get info for all of them:

    images_info = astroquery.get_images_info( access_url=sia_services[:][‘access_url’], pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’]

to get a table list of all images in a list of services that contain that point. Then standardize them as above.



In [25]:
def get_image_url( access_url, pos, size, naxis='',format='fits'):
    """Return a single image url
    
    For some services, you have to specify the format, e.g., SkyView. Is this standard?"""
    # Send the query to get the URLs to any matching images
    result_table=astroquery.standardize(
        astroquery.request2table( url=access_url, params = {'pos': pos, 'size': size, 'naxis':naxis})
    )
    # Find the FITS image url. For SkyView, there will be 2 URLs, one FITS and one JPEG
    return result_table[the_right_one]['url']

def get_images_info( access_urls, pos, size, naxis=''):
    """Return the URLs etc. for images available from a list of services."""
    # Since the columns differ, first get a list of astropy tables.
    results=[] 
    for service_url in access_urls:
        results.append( astroquery.get_image( access_url=service_url, pos=pos,size=size,naxis=naxis) )
    return astroquery.standardize(results)

 

7.

You can look at the images_info and pick one to download:

    astroquery.download_image( images_info[6]['url'], filename='my_file.fits')
    
or get the image data to hand to the plotter:

    image=astroquery.get_image( images_info[6]['url'] )
    plt.imshow( image,  cmap='gray', origin='lower',vmax=0.02 )
    
    



In [ ]:
                                   
def download_image( image_url, filename=''):
    """Download the image and optionally rename """
    urllib.request.urlretrieve(image_url, filename)
    return 

def get_image( image_url ):
    """Returns the data that can be handed to plt.imshow() from a URL"""
    astroquery.download_image( image_url, filename='tmp.fits')
    hdus=astropy.io.fits.open('tmp.fits')
    return hdus[0].data



8

This is complex but can use the above. 

    galex_sia_services = astroquery.list_sia_services( name_like=“galex”) 
    2mass_sia_services = astroquery.list_sia_services( name_like=“2mass”) 
    allwise_sia_services = astroquery.list_sia_services( name_like=“allwise”) 
    
    query_urls=galex_sia_services[:][’access_url’]
    query_urls.extend( 2mass_sia_services[:][’access_url’])
    query_urls.extend( allwise_sia_services[:][’access_url’])
    
    for url in query_urls:
    



    